In [3]:
def blockEtl():
    """
    MariaDB와 OracleDB에 각각 현재 blockchain.csv에 적힌 값들을 저장하고, 저장된 데이터의 무결성을 검증하는 함수입니다.
    총 거래데이터의 길이의 한도를 500자로 한정하였습니다.
    return : targetStr 이라는 문자열 변수를 return하며, 각 케이스에 부합하는 데이터 현황에 관한 정보를 문자열 형태로 출력합니다.    
    """
    # 사용 모듈
    import pandas as pd
    from sqlalchemy import create_engine
    from sqlalchemy import types
    import pandas as pd
    import pickle
    
    # blockchain.csv파일을 읽어와 이를 DataFrame으로 targetData 변수에 저장합니다.
    targetData = pd.read_csv("blockchain.csv")
    
    # genesisBlock의 데이터를 별도로 저장합니다.
    genesisBlock = pd.DataFrame(targetData.columns)
    
    # blockchain.csv 데이터가 담긴 데이터프레임의 컬럼 명을 지정합니다.
    targetData.columns = ["index", "previousHash", "timestamp", "data", "presentHash", "proof"]
    
    # pickle로 저장된 mariaDB 접속 정보를 불러온다.
    with open("mariaData.pickle", "rb") as mariaData:
        mariaDbInfo = pickle.load(mariaData)
    
    mariaId = mariaDbInfo.get("mariaId")
    mariaPw = mariaDbInfo.get("mariaPw")
    mariaIp = mariaDbInfo.get("mariaIp")
    mariaPort = mariaDbInfo.get("mariaPort")
    mariaDbName = mariaDbInfo.get("mariaDbName")
    
    mariaEngine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(mariaId, mariaPw, mariaIp, mariaPort, mariaDbName))

    # pickle로 저장된 oracleDB 접속 정보를 불러온다.
    with open("oracleData.pickle", "rb") as oracleData:
        oracleDbInfo = pickle.load(oracleData)
    
    orclId = oracleDbInfo.get("orclId")
    orclPw = oracleDbInfo.get("orclPw")
    orclIp = oracleDbInfo.get("orclIp")
    orclPort = oracleDbInfo.get("orclPort")
    orclDbName = oracleDbInfo.get("orclDbName")
    
    oracleEngine = create_engine('oracle+cx_oracle://{}:{}@{}:{}/{}'.format(orclId, orclPw, orclIp, orclPort, orclDbName))

    # ETL 간 사용할 테이블명
    tableName = "groupblockdata"

    # 전송 처리 향상을 도모 
    objectColumns = list(targetData.columns[targetData.dtypes == 'object'])
    typeDict={}
    maxLen = 500
    for i in range(0, len(objectColumns)):
        typeDict[ objectColumns[i] ] = types.VARCHAR(maxLen)
    
    # MariaDB와 oracleDB 양 쪽으로 대상 targetData를 전송. 테이블명 : groupblockdata.
    targetData.to_sql(name=tableName, if_exists="replace", con=oracleEngine, dtype=typeDict, index=False)
    targetData.to_sql(name=tableName, if_exists="replace", con=mariaEngine, dtype=typeDict, index=False)

    # MariaDB와 oracleDB로부터 groupblockdata 테이블로부터 데이터를 불러와 각각 mariaBlockData와 oracleBlockData 변수에 저장
    mariaBlockData = pd.read_sql_query("SELECT * FROM groupblockdata", con = mariaEngine)
    oracleBlockData = pd.read_sql_query("SELECT * FROM GROUPBLOCKDATA", con = oracleEngine)
    
    # 두 데이터의 길이가 같다면 targetStr 변수에 True를, 다르다면 False를 저장 
    if(mariaBlockData.shape[0] == oracleBlockData.shape[0]):
        targetStr = "True"
    else:
        targetStr = "False"
    
    # 만약 targetStr이 True라면, 두 테이블의 presentHash 값을 모두 비교하고,
    # 하나라도 다른 해시값이 존재한다면 "온전하지 않은 블록이 존재합니다" 라는 문자열을 return하고 for문을 탈출합니다.
    # 모든 presentHash 값이 일치한다면 "온전한 블록으로 구성되어 있습니다." 라는 문자열을 return합니다.
    if(targetStr == "True"):
        for i in range(0, mariaBlockData.shape[0]):
            if(oracleBlockData.presentHash[i] != mariaBlockData.presentHash[i]):
                answerStr = "온전하지 않은 블록이 존재합니다."
                break
            else:
                answerStr = "온전한 블록으로 구성되어있습니다."
    # 만약 answerStr이 False라면, " 두 DB에 저장된 값이 일치하지 않습니다." 라는 문자열을 return합니다.
    else:
        answerStr = "두 DB에 저장된 값이 일치하지 않습니다."

    return answerStr